# End-to-end Exploratory Data Analisys

Neste notebook, vamos analisar um conjunto de dados sobre a taxa de rotatividade dos clientes das operadoras de telecomunicações. Vamos rever alguns métodos e atributos estudados bem como aprender novos. Além disso, vamos plotar gráficos para entender de maneira visual como nossos dados se comportam e que informações podemos extrair deles.

In [ ]:
# importanto as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Fazendo a leitura dos dados
df = pd.read_csv('bases/telecom_churn.csv')
df.head()

Cada linha desse dataset é uma instância (amostra) e cada coluna é uma feature (característica). Um algoritmo de machine learning poderia ser aplicado aqui para, a partir desse conjunto de features, prever a variável churn (variável meta). 

In [ ]:
# Vamos analisar a dimensionalidade e o tipo de cada característica presente no dataset
print(df.shape)
print('-------------------------')
print(df.info())

Podemos perceber que não existem valores faltantes no dataset. Entretando, ao verificar o tipo de dado de cada feature, observamos que Churn é uma coluna booleana. Entretanto, valores booleanos não são aceitos como valores de entrada para algoritmos de machine learning. Vamos tratar isso:

In [ ]:
df['Churn'] = df['Churn'].astype('int64')

Agora podemos executar o método describe para analisar as estatísticas básicas do dataset

In [ ]:
df.describe()

Podemos analisar algumas estatísticas de features não numéricas tornando isso explicito ao chamar o método describe 

In [ ]:
df.describe(include=['object', 'bool'])

Ainda podemos usar value_counts() para analisar valores categóricos

In [ ]:
df['Churn'].value_counts()

Se eu quiser que esses valores sejam representados na forma de porcentagem, posso usar o parâmetro normalize

In [ ]:
df['Churn'].value_counts(normalize=True)

Podemos ordenar os valores de uma coluna usando o método sort_values()

In [ ]:
df.sort_values(by='Total day charge', ascending=False).head()

É possível, também, ordenar o dataset usando várias colunas

In [ ]:
df.sort_values(by=['Churn', 'Total day charge'],ascending=True).head()

Indexação é uma importante ferramenta que nos permite descobrir informações muito relevantes num dataset. Esse é o processo de indexação:

In [ ]:
# proporção de clientes que rotacionaram
df['Churn'].mean()

Com isso em mente, podemos querer fazer uma comparação das características médias dos clientes que rotacionaram e daqueles que não rotacionaram. Vejamos:

In [ ]:
#rotacionaram
df[df['Churn'] == 1].mean()

In [ ]:
# não rotacionaram
df[df['Churn'] == 0].mean()

Além disso, podemos responder outros questionamentos:

In [ ]:
# Quanto tempo (em média) usuários que rotacionaram gastam no telefone durante o dia?
df[df['Churn'] == 1]['Total day minutes'].mean()

In [ ]:
# Qual a duração máxima das chamadas internacionais entre usuários fiéis (rotatividade == 0) que não possuem um plano internacional?
df[(df['Churn'] == 0) & (df['International plan'] == 'No')]['Total intl minutes'].max()

O método loc nos permite indexar o dataset a fim de trazer apenas informações de interesse. Observe:

In [ ]:
# 5 primeiras linhas das colunas especificadas
df.loc[0:5, 'State':'Area code']

In [ ]:
# 5 primeiras linhas das 3 primeiras colunas
df.iloc[0:5, 0:3]

Podemos usar apply para aplicar funções numa coluna (vetorização):

In [ ]:
df.apply(np.max)

Podemos usar apply para, por exemplo, trazer informações de todos os estados que começam com a letra w:

In [ ]:
df[df['State'].apply(lambda state: state[0] == 'W')].head()

Podemos uar o map para substituir mais de um valor de uma vez por valores correspondentes numa determinada coluna. Vejamos:

In [ ]:
d = {'No' : False, 'Yes' : True}
df['International plan'] = df['International plan'].map(d)
df.head()

O mesmo pode ser feito utilizando o método replace():

In [ ]:
df = df.replace({'Voice mail plan': d})
df.head()

O agrupamento de dados funciona de acordo com o seguinte esquema: 

df.groupby(by=grouping_columns)[columns_to_show].function()

> primeiro, o método groupby divide as grouping_columns por seus valores. 

> depois, as colunas de interesse são selecionadas (columns_to_show). Se columns_to_show não for incluída, tudo o que não estiver no groupby será incluído

> por fim, uma ou mais funções podem ser aplicadas aos grupos


Observe um exemplo:

In [ ]:
columns_to_show = ['Total day minutes', 'Total eve minutes', 
                   'Total night minutes']

df.groupby(['Churn'])[columns_to_show].agg([np.mean, np.std, np.min, 
                                            np.max])

# Visualização de Dados

In [ ]:
df.head()

In [ ]:
# Distribuição de clientes que rotacionaram
churn = [0,0]
churn[0] = len(df[df['Churn']==0])
churn[1] = len(df[df['Churn']==1])
plt.figure(figsize=(15,5))
plt.bar(['False', 'True'],churn)

In [ ]:
x = df[df['Churn'] == 0]
x = x[x['International plan'] == False]

y = df[df['Churn'] == 0]
y = y[y['International plan'] == True]

z = df[df['Churn'] == 1]
z = z[z['International plan'] == False]

w = df[df['Churn'] == 1]
w = w[w['International plan'] == True]

In [ ]:
# Rotatividade de acordo com Plano internacional
N = 2
plan0 = (len(x), len(y))
plan1 = (len(z), len(w))
ind = np.arange(2)    
width = 0.35       # largura das barras

p1 = plt.bar(ind, plan0, width)
p2 = plt.bar(ind, plan1, width,bottom=plan0)

plt.ylabel('Contagem')
plt.title('Rotatividade de acordo com plano internacional')
plt.xticks(ind, ('Não tem', 'tem'))
plt.legend((p1[0], p2[0]), ('Não Rotativo', 'Rotativo'))

plt.show()